# План занятия
 - Локальные БД. Работа с библиотекой sqlite3
 - Представление БД с использованием Pandas
 - Удаленные БД. Работа с библиотекой SQLAlchemy

https://colab.research.google.com/drive/16bFKpKclPqjvJ18yvFAZQW9fx73U5Fbi?usp=sharing

# Введение

База данных может храниться локально, на том же компьютере, на котором запускается прикладное программное обеспечение для работы с базой данных или на удаленном компьютере.

Google Colab (точнее Python, запускаемый в среде Google Colab) позволяет работать и с условно «локальной» базой данных и с базой данных на удаленном сервере. 
- В первом случае может использоваться СУБД **SQLite**, которая будет хранится на облачном диске Google Drive. Для среды Google Colab это будет локальная БД.
- В случае работы с удаленной БД можно использовать **SQLAlchemy**.

Оба модуля **SQLite** и **SQLAlchemy** рассмотрим ниже.

# SQLite

**SQLite** — компактная встраиваемая реляционная база данных. Является чисто реляционной базой данных.

Слово «встраиваемый» означает, что SQLite **не использует парадигму клиент-сервер.** Модуль sqlite3 входит в установочный пакет языка Python, является компонентом стандартной библиотеки и не требует отдельной загрузки и установки.

Pipeline для работы с БД при помощи библиотеки SQLite

``` 
import sqlite3

conn = sqlite3.connect(path)

cursor = conn.cursor()

# Работа с БД
# ...
# ...

cursor.close()
conn.close()
```

## Подключение к базе данных

Для доступа к диску Google Drive сервису Google Colab необходимо дать разрешение на подключение к диску.

Для подключения и разрешения доступа Google Colab к диску Google Drive используем следующий код:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Подключаем модуль для работы с базой SQLite

In [ ]:
import sqlite3

Непосредственно модуль sqlite3 – это API к СУБД SQLite. Своего рода адаптер, который переводит команды, написанные на Питоне, в команды, которые понимает SQLite. Как и наоборот, доставляет ответы от SQLite в python-программу.

In [ ]:
dir(sqlite3)

['Binary',
 'Cache',
 'Connection',
 'Cursor',
 'DataError',
 'DatabaseError',
 'Date',
 'DateFromTicks',
 'Error',
 'IntegrityError',
 'InterfaceError',
 'InternalError',
 'NotSupportedError',
 'OperationalError',
 'OptimizedUnicode',
 'PARSE_COLNAMES',
 'PARSE_DECLTYPES',
 'PrepareProtocol',
 'ProgrammingError',
 'Row',
 'SQLITE_ALTER_TABLE',
 'SQLITE_ANALYZE',
 'SQLITE_ATTACH',
 'SQLITE_CREATE_INDEX',
 'SQLITE_CREATE_TABLE',
 'SQLITE_CREATE_TEMP_INDEX',
 'SQLITE_CREATE_TEMP_TABLE',
 'SQLITE_CREATE_TEMP_TRIGGER',
 'SQLITE_CREATE_TEMP_VIEW',
 'SQLITE_CREATE_TRIGGER',
 'SQLITE_CREATE_VIEW',
 'SQLITE_DELETE',
 'SQLITE_DENY',
 'SQLITE_DETACH',
 'SQLITE_DROP_INDEX',
 'SQLITE_DROP_TABLE',
 'SQLITE_DROP_TEMP_INDEX',
 'SQLITE_DROP_TEMP_TABLE',
 'SQLITE_DROP_TEMP_TRIGGER',
 'SQLITE_DROP_TEMP_VIEW',
 'SQLITE_DROP_TRIGGER',
 'SQLITE_DROP_VIEW',
 'SQLITE_IGNORE',
 'SQLITE_INSERT',
 'SQLITE_OK',
 'SQLITE_PRAGMA',
 'SQLITE_READ',
 'SQLITE_REINDEX',
 'SQLITE_SELECT',
 'SQLITE_TRANSACTION',
 'SQLITE

Создадим папку "my_databases", где будут хранится наши БД

In [ ]:
import os
path = "./drive/My Drive/my_databases"
if not os.path.exists(path):
    os.mkdir(path)

In [ ]:
os.path.exists(path)

True

Подключаемся к базе test.db. Если этого файла нет в каталоге, то он будет создан.

Вызов функции connect() приводит к созданию объекта-экземпляра от класса Connection. Этот объект обеспечивает связь с файлом базы данных, представляет конкретную БД в программе:

In [ ]:
conn = sqlite3.connect(path + '/test.db')
print("Opened database successfully");

Opened database successfully


После того как экземпляр Connection создан, чтобы выполнять SQL-команды, надо создать еще один объект, но теперь уже от класса Cursor. Делается это с помощью метода cursor() объекта типа Connection:

In [ ]:
cursor = conn.cursor()

In [ ]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS team_data(team text, 
                      country text, 
                      season integer, 
                      total_goals integer);''')

conn.commit()

print("Table created successfully");

#conn.close()

Table created successfully


Заполнять таблицы можно тоже с помощью **execute().**

In [ ]:
# INSERTING VALUES

cursor.execute("INSERT INTO team_data VALUES('Real Madrid', 'Spain', 2019, 53);")
cursor.execute("INSERT INTO team_data VALUES('Barcelona', 'Spain', 2019, 47);")

Однако, если требуется вставить несколько записей, лучше воспользоваться методом **executemany():**

In [ ]:
sections = [('Arsenal', 'UK', 2019, 52), ('Real Madrid', 'Spain', 2018, 49),
             ('Barcelona', 'Spain', 2018, 45), ('Arsenal', 'UK', 2018, 50 )]
cursor.executemany("INSERT INTO team_data VALUES (?, ?, ?, ?)", sections)

Для того, чтобы корректно завершить работу с базой данных, надо применить изменения (выполнить транзакцию) **commit()** и разорвать соединение **close()**. Обратите внимание, это делается по отношению к экземпляру Connection, а не Cursor:

In [ ]:
conn.commit()
conn.close()

In [ ]:
# Average goal by team

conn = sqlite3.connect(path + '/test.db')
# Создаем объект типа cursor для доступа к данным
cursor = conn.cursor()

#Запрос данных из таблицы 
sql  = '''SELECT team, total_goals FROM team_data'''

cursor.execute(sql)

for row in cursor:
    print(row)

('Real Madrid', 53)
('Barcelona', 47)
('Arsenal', 52)
('Real Madrid', 49)
('Barcelona', 45)
('Arsenal', 50)
('Real Madrid', 53)
('Barcelona', 47)
('Arsenal', 52)
('Real Madrid', 49)
('Barcelona', 45)
('Arsenal', 50)


In [ ]:
#Запрос данных из таблицы 
sql  = ''' SELECT team, AVG(total_goals) FROM team_data'''

cursor.execute(sql)

for row in cursor:
    print(row)

('Arsenal', 49.333333333333336)


Почему на выходе получили только 1 комманду?

In [ ]:
sql  = ''' SELECT team, AVG(total_goals) AS avg_goals FROM team_data GROUP BY team;'''
cursor.execute(sql)

for row in cursor:
  print(row)

ProgrammingError: ignored

In [ ]:
cursor.close()
conn.close()

ProgrammingError: ignored

In [ ]:
# First try to filter the teams with average goals higher than 50
# This query will generate an error

conn = sqlite3.connect(path + '/test.db')
cursor = conn.cursor()

sql = ''' SELECT team AS team_name,
                            AVG(total_goals) AS avg_goals
                          FROM team_data
                          GROUP BY team 
                          HAVING AVG(total_goals) > 50;'''
                          
cursor.execute(sql)

for row in cursor:
  print(row)
conn.close()

('Arsenal', 51.0)
('Real Madrid', 51.0)


In [ ]:
# Now, the correct query, using the appropriate sub-query

conn = sqlite3.connect(path + '/test.db')

cursor = conn.cursor()

sql = ''' SELECT team_name, avg_goals
                          FROM (

                          -- Here we make our sub-query:
                            SELECT team AS team_name,
                            AVG(total_goals) AS avg_goals
                            FROM team_data
                            GROUP BY team) tp
                          -- End of the sub-query
                          
                          WHERE avg_goals > 50;'''

cursor.execute(sql)

for row in cursor:
  print(row)
conn.close()

('Arsenal', 51.0)
('Real Madrid', 51.0)


Для наглядного представления табличных данных можно использовать библиотеку pandas:

In [ ]:
import pandas as pd


conn = sqlite3.connect(path + '/test.db')
cursor = conn.cursor()

#sql  = '''SELECT team, total_goals AS avg_goals FROM team_data GROUP BY team;'''
sql  = '''SELECT team, total_goals FROM team_data;'''
                          
cursor.execute(sql)

# Загружаем все результаты в список списков rows 
rows = cursor.fetchall()

In [ ]:
rows

[('Real Madrid', 53),
 ('Barcelona', 47),
 ('Arsenal', 52),
 ('Real Madrid', 49),
 ('Barcelona', 45),
 ('Arsenal', 50),
 ('Real Madrid', 53),
 ('Barcelona', 47),
 ('Arsenal', 52),
 ('Real Madrid', 49),
 ('Barcelona', 45),
 ('Arsenal', 50)]

In [ ]:
pd.DataFrame(rows, columns=('Team', 'Goals') )

,Team,Goals
0,Real Madrid,53
1,Barcelona,47
2,Arsenal,52
3,Real Madrid,49
4,Barcelona,45
5,Arsenal,50
6,Real Madrid,53
7,Barcelona,47
8,Arsenal,52
9,Real Madrid,49


In [ ]:
sql  = '''SELECT team, total_goals AS avg_goals FROM team_data GROUP BY team;'''
                          
cursor.execute(sql)

# Загружаем все результаты в список списков rows 
rows = cursor.fetchall()

pd.DataFrame(rows, columns=('Team', 'Goals'))

,Team,Goals
0,Arsenal,50
1,Barcelona,45
2,Real Madrid,49


In [ ]:
cursor.close()
conn.close()

<center><img src='https://raw.githubusercontent.com/ddvika/Data-Science-School-2020/main/lecture_5/imgs/typical_sql.tiff' height = 550></center>

# SQLAlchemy

**ORM** расшифровывается как object-relational mapping,  или объектно-реляционное отображение — подход к работе с базами данных, использующий  концепции объектно-ориентированных языков программирования.   Объектно-реляционное отображение позволяет оперировать объектами в коде, что гораздо удобнее, чем работать с запросами и таблицами.


Иными словами, можно обращаться к объектам классов для управления данными в таблицах БД. Также можно создавать, изменять, удалять, фильтровать и, самое главное, наследовать объекты классов, сопоставленные с таблицами БД, что существенно сокращает наполнение кодовой базы.

**SQLAlchemy** — это библиотека на языке Python для работы с реляционными СУБД с применением технологии ORM. Служит для синхронизации объектов Python и записей реляционной базы данных. SQLAlchemy позволяет описывать структуры баз данных и способы взаимодействия с ними на языке Python без использования SQL.

### Создадим новую базу данных с нуля
Давайте создадим новую базу данных с нуля, для этого:
1. Создадим классы для определения схемы.
2. Сопоставим схему с базой данных.
3. Добавим объекты в базу данных
4. Напишем запросы

### 1. Запустим database session

In [ ]:
from sqlalchemy import create_engine
#engine = create_engine('sqlite:///example.db', echo=True)
engine = create_engine('sqlite:///:memory:', echo=True)
#engine = create_engine('sqlite:///:memory:')
conn = engine.connect()

from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

2020-12-10 12:36:52,544 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-10 12:36:52,546 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 12:36:52,549 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-10 12:36:52,551 INFO sqlalchemy.engine.base.Engine ()


В случае если мы хоти подкоючится к удаленной БД, код будет выглядить примерно так:

```
def connect_to_db(uid):
    # создаем объект подключения
    sql = create_engine(
        f'mysql+mysqlconnector://user{uid}:userpassword{uid}'   # LOGIN, PASSWORD
        f'@157.230.109.1/classicmodels_user_{uid}',             # HOST IP
        pool_recycle=60
    )
    connection = sql.connect()
    return connection
    # отправка комманд без чтения/записи (например удаление) в самом конце

connection = connect_to_db(uid=5)
connection
```

### 2. Вспомогательные функции для печати и вывода результатов SQL запросов

In [ ]:
from IPython.display import display
import pandas as pd
import sqlalchemy

def sql(query):
    print()
    print(query)
    print()

def get_results(query):
    global engine
    q = query.statement if isinstance(query, sqlalchemy.orm.query.Query) else query
    return pd.read_sql(q, engine)

def display_results(query):
    df = get_results(query)
    display(df)
    #sql(query)

### 3. Инициализация схемы БД

Схема - это пространство имен, которое содержит именованные объекты базы данных, такие как таблицы, представления, индексы, типы данных

In [ ]:
!pip install sqlalchemy_explore

  Created wheel for sqlalchemy-explore: filename=sqlalchemy_explore-0.1.2-cp36-none-any.whl size=4546 sha256=a000943424895e360ba7286346ad65c92318e42ce3b502dc2dd9bb88ce22f43a
  Stored in directory: /root/.cache/pip/wheels/af/7d/67/1c55901ebff236da60829800b9a2ebfa8b52c455db565a1acd
Successfully built sqlalchemy-explore


In [ ]:
from sqlalchemy.ext.declarative import declarative_base
import sqlalchemy_explore

### the basic base class for SQLAlchemy schema objects
# Base = declarative_base(bind=engine)

### base class including utils like an __repr__ method
### see https://pypi.org/project/sqlalchemy-explore/
Base = declarative_base(cls=sqlalchemy_explore.ReflectiveMixin)

### Создание самой схемы

In [ ]:
from sqlalchemy import Column, DateTime, ForeignKey, Integer, NVARCHAR, Numeric, Sequence
from sqlalchemy.orm import relationship

class Customer(Base):
    __tablename__ = 'customers'

    CustomerId = Column(Integer, Sequence('customer_id_seq'), primary_key=True)
    FirstName = Column(NVARCHAR(40), nullable=False)
    LastName = Column(NVARCHAR(20), nullable=False)
    Company = Column(NVARCHAR(80))
    Address = Column(NVARCHAR(70))
    Phone = Column(NVARCHAR(24))
    Email = Column(NVARCHAR(60), nullable=False)
    
class Item(Base):
    __tablename__ = 'items'
    
    ItemId = Column(Integer, Sequence('item_id_seq'), primary_key=True)
    Name = Column(NVARCHAR(40), nullable=False)
    Price = Column(Numeric, nullable=False)

class Purchase(Base):
    __tablename__ = 'purchases'
    
    PurchaseId = Column(Integer, Sequence('purchase_id_seq'), primary_key=True)
    ItemId = Column(ForeignKey('items.ItemId'), nullable=False, index=True)
    CustomerId = Column(ForeignKey('customers.CustomerId'), nullable=False, index=True)
    Date = Column(DateTime, nullable=False)
    
    item = relationship('Item')
    customer = relationship('Customer')

In [ ]:
Purchase.ItemId.name

'ItemId'

In [ ]:
Purchase.CustomerId.name

'CustomerId'

### 5. Создадим таблицы в базе данных в соответствии со схемой

In [ ]:
Base.metadata.create_all(engine)

2020-12-10 12:37:46,266 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("customers")
2020-12-10 12:37:46,269 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 12:37:46,271 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("customers")
2020-12-10 12:37:46,273 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 12:37:46,275 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("items")
2020-12-10 12:37:46,276 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 12:37:46,277 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("items")
2020-12-10 12:37:46,278 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 12:37:46,280 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("purchases")
2020-12-10 12:37:46,281 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 12:37:46,282 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("purchases")
2020-12-10 12:37:46,283 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 12:37:46,285 INFO sqlalchemy.engine.base.Engine 
CREATE

In [ ]:
engine.table_names()

2020-12-10 12:37:51,504 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-12-10 12:37:51,505 INFO sqlalchemy.engine.base.Engine ()


['customers', 'items', 'purchases']

### 6. Создадим покупателя

In [ ]:
moshe = Customer(
    FirstName='Moshe', 
    LastName='Cohen', 
    Address='Alenbi 99, Tel Aviv', 
    Phone="053-5556789", 
    Email='moshe@cohen.com')

session.add(moshe)
session.commit()

2020-12-10 12:44:28,792 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-10 12:44:28,797 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2020-12-10 12:44:28,799 INFO sqlalchemy.engine.base.Engine ('Moshe', 'Cohen', None, 'Alenbi 99, Tel Aviv', '053-5556789', 'moshe@cohen.com')
2020-12-10 12:44:28,802 INFO sqlalchemy.engine.base.Engine COMMIT


### 7. Выполним запрос

Используя язык выражений SQLAchemy

In [ ]:
from sqlalchemy import select 

customers_query = select([Customer.FirstName, Customer.LastName, Customer.Email])
results = conn.execute(customers_query)

print()
for row in results:
    print(row)

print()
print(type(row)) # rows are of type sqlalchemy.engine.result.RowProxy

2020-12-10 12:37:59,323 INFO sqlalchemy.engine.base.Engine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers
2020-12-10 12:37:59,324 INFO sqlalchemy.engine.base.Engine ()

('Moshe', 'Cohen', 'moshe@cohen.com')

<class 'sqlalchemy.engine.result.RowProxy'>


In [ ]:
display_results(customers_query)

2020-12-10 12:38:04,412 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers
2020-12-10 12:38:04,413 INFO sqlalchemy.engine.base.OptionEngine ()


,FirstName,LastName,Email
0,Moshe,Cohen,moshe@cohen.com


### 8. Добавим еще покупателей

In [ ]:
Lisa = Customer(
    FirstName='Lisa',
    LastName='Cohen', 
    Address='Alenbi 99, Tel Aviv', 
    Phone="052-1234565", 
    Email='lisa@cohen.com')

session.add(Lisa)
session.commit()

2020-12-10 12:38:08,724 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-10 12:38:08,729 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2020-12-10 12:38:08,732 INFO sqlalchemy.engine.base.Engine ('Lisa', 'Cohen', None, 'Alenbi 99, Tel Aviv', '052-1234565', 'lisa@cohen.com')
2020-12-10 12:38:08,733 INFO sqlalchemy.engine.base.Engine COMMIT


In [ ]:
Nika = Customer(
    FirstName='Nika', 
    LastName='Rave', 
    Address='Green st, LA', 
    Phone="330-1234565", 
    Email='Nika@rave.com')

session.add(Nika)
session.commit()

2020-12-10 12:44:07,836 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2020-12-10 12:44:07,838 INFO sqlalchemy.engine.base.Engine ('Nika', 'Rave', None, 'Green st, LA', '330-1234565', 'Nika@rave.com')
2020-12-10 12:44:07,840 INFO sqlalchemy.engine.base.Engine COMMIT


In [ ]:
Lisa_2 = Customer(
    FirstName='Lisa',
    LastName='White', 
    Address='Alenbi 66, Tel Aviv', 
    Phone="062-1234565", 
    Email='lisa@White.com')

session.add(Lisa_2)
session.commit()

2020-12-10 12:44:11,069 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-10 12:44:11,071 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2020-12-10 12:44:11,072 INFO sqlalchemy.engine.base.Engine ('Lisa', 'White', None, 'Alenbi 66, Tel Aviv', '062-1234565', 'lisa@White.com')
2020-12-10 12:44:11,074 INFO sqlalchemy.engine.base.Engine COMMIT


Посмотрим теперь на таблицу:

In [ ]:
customers_query = select([Customer.FirstName, Customer.Email])
display_results(customers_query)

2020-12-10 12:38:23,751 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."Email" 
FROM customers
2020-12-10 12:38:23,755 INFO sqlalchemy.engine.base.OptionEngine ()


,FirstName,Email
0,Moshe,moshe@cohen.com
1,Lisa,lisa@cohen.com
2,Nika,Nika@rave.com
3,Lisa,lisa@White.com


In [ ]:
Customer.__dict__

mappingproxy({'Address': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7f378d8c5258>,
              'Company': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7f378d8c52b0>,
              'CustomerId': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7f3771f10fc0>,
              'Email': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7f378d8c50a0>,
              'FirstName': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7f378d91c830>,
              'LastName': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7f378d8c5308>,
              'Phone': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7f378d8c5200>,
              '__doc__': None,
              '__init__': <function __init__>,
              '__mapper__': <Mapper at 0x7f3771f2c7f0; Customer>,
              '__module__': '__main__',
              '__table__': Table('customers', MetaData(bind=None), Column('CustomerId', Integer(), table=<customers>, primary_key=True, nullable=False, defa

In [ ]:
list_info = session.query(Customer).all()

2020-12-10 12:38:32,798 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-10 12:38:32,802 INFO sqlalchemy.engine.base.Engine SELECT customers."CustomerId" AS "customers_CustomerId", customers."FirstName" AS "customers_FirstName", customers."LastName" AS "customers_LastName", customers."Company" AS "customers_Company", customers."Address" AS "customers_Address", customers."Phone" AS "customers_Phone", customers."Email" AS "customers_Email" 
FROM customers
2020-12-10 12:38:32,808 INFO sqlalchemy.engine.base.Engine ()


In [ ]:
list_info

[Customer(CustomerId=1, FirstName='Moshe', LastName='Cohen', Company=None, Address='Alenbi 99, Tel Aviv', Phone='053-5556789', Email='moshe@cohen.com'),
 Customer(CustomerId=2, FirstName='Lisa', LastName='Cohen', Company=None, Address='Alenbi 99, Tel Aviv', Phone='052-1234565', Email='lisa@cohen.com'),
 Customer(CustomerId=3, FirstName='Nika', LastName='Rave', Company=None, Address='Green st, LA', Phone='330-1234565', Email='Nika@rave.com'),
 Customer(CustomerId=4, FirstName='Lisa', LastName='White', Company=None, Address='Alenbi 66, Tel Aviv', Phone='062-1234565', Email='lisa@White.com')]

In [ ]:
for id_ in list_info:
    print(id_.CustomerId)

1
2
3
4


In [ ]:
first_info = session.query(Customer.CustomerId)

In [ ]:
first_info

Операторы **desc**, **asc**, **order_by**

Оператор SQL ORDER BY выполняет сортировку выходных значений. Оператор SQL ORDER BY можно применять как к числовым столбцам, так и к строковым. В последнем случае, сортировка будет происходить по алфавиту.

`ORDER BY column_name [ASC | DESC]`

ASC - по возрастанию, DESC - по убыванию

In [ ]:
from sqlalchemy import *

In [ ]:
from sqlalchemy import desc, asc

customers_query = select([Customer.FirstName, Customer.LastName, Customer.Email]).order_by(asc(Customer.FirstName), desc(Customer.LastName))
#customers_query = customers_query.order_by(asc(Customer.FirstName))

display_results(customers_query)

2020-12-10 12:38:52,741 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers ORDER BY customers."FirstName" ASC, customers."LastName" DESC
2020-12-10 12:38:52,744 INFO sqlalchemy.engine.base.OptionEngine ()


,FirstName,LastName,Email
0,Lisa,White,lisa@White.com
1,Lisa,Cohen,lisa@cohen.com
2,Moshe,Cohen,moshe@cohen.com
3,Nika,Rave,Nika@rave.com


In [ ]:
customers_query = customers_query.order_by(desc(Customer.FirstName))

display_results(customers_query)

2020-12-10 12:38:58,785 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers ORDER BY customers."FirstName" ASC, customers."LastName" DESC, customers."FirstName" DESC
2020-12-10 12:38:58,787 INFO sqlalchemy.engine.base.OptionEngine ()


,FirstName,LastName,Email
0,Lisa,White,lisa@White.com
1,Lisa,Cohen,lisa@cohen.com
2,Moshe,Cohen,moshe@cohen.com
3,Nika,Rave,Nika@rave.com


**where**
Оператор SQL WHERE служит для задания дополнительного условия выборки, операций вставки, редактирования и удаления записей.

`where condition`

In [ ]:
customers_query = select([Customer.FirstName, Customer.LastName, Customer.Email])
customers_query = customers_query.where(Customer.FirstName == 'Lisa')

display_results(customers_query)

2020-12-10 12:39:10,496 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers 
WHERE customers."FirstName" = ?
2020-12-10 12:39:10,498 INFO sqlalchemy.engine.base.OptionEngine ('Lisa',)


,FirstName,LastName,Email
0,Lisa,Cohen,lisa@cohen.com
1,Lisa,White,lisa@White.com


In [ ]:
customers_query = select([Customer.FirstName, Customer.LastName, Customer.Email])
customers_query = customers_query.where(Customer.FirstName == 'Lisa').order_by(asc(Customer.LastName))

In [ ]:
display_results(customers_query)

2020-12-10 12:39:23,677 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers 
WHERE customers."FirstName" = ? ORDER BY customers."LastName" ASC
2020-12-10 12:39:23,679 INFO sqlalchemy.engine.base.OptionEngine ('Lisa',)


,FirstName,LastName,Email
0,Lisa,Cohen,lisa@cohen.com
1,Lisa,White,lisa@White.com


Документация по операторам в SQLAchemy:

https://docs.sqlalchemy.org/en/13/core/sqlelement.html

## SQLAchemy + Pandas

In [ ]:
import pandas as pd

In [ ]:
#выведем название нашей таблицы
Item.__tablename__

'items'

In [ ]:
df = pd.read_sql('SELECT * FROM customers', conn)
df.tail()

2020-12-10 12:39:40,525 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("SELECT * FROM customers")
2020-12-10 12:39:40,526 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 12:39:40,528 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("SELECT * FROM customers")
2020-12-10 12:39:40,530 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 12:39:40,531 INFO sqlalchemy.engine.base.Engine SELECT * FROM customers
2020-12-10 12:39:40,532 INFO sqlalchemy.engine.base.Engine ()


,CustomerId,FirstName,LastName,Company,Address,Phone,Email
0,1,Moshe,Cohen,None,"Alenbi 99, Tel Aviv",053-5556789,moshe@cohen.com
1,2,Lisa,Cohen,None,"Alenbi 99, Tel Aviv",052-1234565,lisa@cohen.com
2,3,Nika,Rave,None,"Green st, LA",330-1234565,Nika@rave.com
3,4,Lisa,White,None,"Alenbi 66, Tel Aviv",062-1234565,lisa@White.com


In [ ]:
# Если хотим получить отсортированные значения - ключевая команда ORDER BY
sql = """
SELECT 
    *
FROM
    customers
ORDER BY 
    LastName ASC;
"""
df = pd.read_sql(sql, conn)

df.head()

2020-12-10 12:39:45,735 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("
SELECT 
    *
FROM
    customers
ORDER BY 
    LastName ASC;
")
2020-12-10 12:39:45,738 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 12:39:45,740 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("
SELECT 
    *
FROM
    customers
ORDER BY 
    LastName ASC;
")
2020-12-10 12:39:45,745 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 12:39:45,747 INFO sqlalchemy.engine.base.Engine 
SELECT 
    *
FROM
    customers
ORDER BY 
    LastName ASC;

2020-12-10 12:39:45,748 INFO sqlalchemy.engine.base.Engine ()


,CustomerId,FirstName,LastName,Company,Address,Phone,Email
0,1,Moshe,Cohen,None,"Alenbi 99, Tel Aviv",053-5556789,moshe@cohen.com
1,2,Lisa,Cohen,None,"Alenbi 99, Tel Aviv",052-1234565,lisa@cohen.com
2,3,Nika,Rave,None,"Green st, LA",330-1234565,Nika@rave.com
3,4,Lisa,White,None,"Alenbi 66, Tel Aviv",062-1234565,lisa@White.com


In [ ]:
# Если хотим получить уникальные значения - ключевая команда DISTINCT
df = pd.read_sql("""
SELECT 
    DISTINCT LastName
FROM
    customers
ORDER BY 
    LastName;
""", conn)

df.head()

2020-12-10 12:39:54,678 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("
SELECT 
    DISTINCT LastName
FROM
    customers
ORDER BY 
    LastName;
")
2020-12-10 12:39:54,679 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 12:39:54,680 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("
SELECT 
    DISTINCT LastName
FROM
    customers
ORDER BY 
    LastName;
")
2020-12-10 12:39:54,681 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 12:39:54,683 INFO sqlalchemy.engine.base.Engine 
SELECT 
    DISTINCT LastName
FROM
    customers
ORDER BY 
    LastName;

2020-12-10 12:39:54,684 INFO sqlalchemy.engine.base.Engine ()


,LastName
0,Cohen
1,Rave
2,White


# ДЗ
 Дедлайн **15.12.2020, 23:59**
  
- Решить первые 20 задач с сайта https://www.sql-ex.ru (Раздел SELECT(обучающий этап)). Оценивается пропорционально кол-ву выполненных задач. + 20 % (еще 15 задач)

- Дополнить таблицу 'items' и 'purchases' 5ью экземплярами (каждую) на Ваш Выбор. Важно: они должны быть привязаны к существующим покупателям Lisa, Nika, Moshe.
- При помощи запроса SQLAlchemy вывести все покупки Lisa. Вывести тоже самое при помощи Pandas.

In [ ]:
customers_query = select([Customer.FirstName, Customer.LastName, Customer.Email])
#customers_query = customers_query.where(Customer.FirstName == 'Lisa').order_by(asc(Customer.LastName))
display_results(customers_query)

2020-12-10 12:48:13,276 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers
2020-12-10 12:48:13,281 INFO sqlalchemy.engine.base.OptionEngine ()


,FirstName,LastName,Email
0,Moshe,Cohen,moshe@cohen.com
1,Lisa,Cohen,lisa@cohen.com
2,Nika,Rave,Nika@rave.com
3,Lisa,White,lisa@White.com
4,Nika,Rave,Nika@rave.com
5,Lisa,White,lisa@White.com
6,Moshe,Cohen,moshe@cohen.com
7,Nika,Rave,Nika@rave.com


Дополнить таблицу 'items' и 'purchases' 5ью экземплярами (каждую) на Ваш Выбор. Важно: они должны быть привязаны к существующим покупателям Lisa, Nika, Moshe.

In [ ]:
Auto1 = Item(
    Name = 'Audi 1',
    Price = 1000)
Auto2 = Item(
    Name = 'Audi 2',
    Price = 2000)
Auto3 = Item(
    Name = 'Audi 3',
    Price = 3000)
Auto4 = Item(
    Name = 'Audi 4',
    Price = 4000)
Auto5 = Item(
    Name = 'Audi 5',
    Price = 5000)

session.add(Auto1)
session.add(Auto2)
session.add(Auto3)
session.add(Auto4)
session.add(Auto5)
session.commit()


2020-12-10 12:58:31,809 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-10 12:58:31,812 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-10 12:58:31,815 INFO sqlalchemy.engine.base.Engine ('Audi 1', 1000.0)
2020-12-10 12:58:31,818 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-10 12:58:31,819 INFO sqlalchemy.engine.base.Engine ('Audi 2', 2000.0)
2020-12-10 12:58:31,820 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-10 12:58:31,823 INFO sqlalchemy.engine.base.Engine ('Audi 3', 3000.0)
2020-12-10 12:58:31,825 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-10 12:58:31,827 INFO sqlalchemy.engine.base.Engine ('Audi 4', 4000.0)
2020-12-10 12:58:31,828 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-10 12:58:31,831 INFO sqlalchemy.engine.base.Engine ('Audi 5

In [ ]:
item_query =  select([Item.Name, Item.Price])  
display_results(item_query)

2020-12-10 14:34:44,715 INFO sqlalchemy.engine.base.OptionEngine SELECT items."Name", items."Price" 
FROM items
2020-12-10 14:34:44,716 INFO sqlalchemy.engine.base.OptionEngine ()


,Name,Price
0,Audi 1,1000.0
1,Audi 2,2000.0
2,Audi 3,3000.0
3,Audi 4,4000.0
4,Audi 5,5000.0


In [ ]:
import datetime
import time
P1 = Purchase(
    ItemId = 6,
CustomerId = 1,
Date = datetime.datetime.now())
time.sleep(1)
P2 = Purchase(
    ItemId = 7,
CustomerId = 2,
Date = datetime.datetime.now())
time.sleep(1)
P3 = Purchase(
    ItemId = 5,
CustomerId = 3,
Date = datetime.datetime.now())
time.sleep(1)
P4 = Purchase(
    ItemId = 3,
CustomerId = 4,
Date = datetime.datetime.now())
time.sleep(1)
P5 = Purchase(
    ItemId = 4,
CustomerId = 6,
Date = datetime.datetime.now())
time.sleep(1)

session.add(P1)
session.add(P2)
session.add(P3)
session.add(P4)
session.add(P5)
session.commit()

2020-12-10 13:18:41,548 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-10 13:18:41,549 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-10 13:18:41,551 INFO sqlalchemy.engine.base.Engine (6, 1, '2020-12-10 13:18:36.541021')
2020-12-10 13:18:41,553 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-10 13:18:41,555 INFO sqlalchemy.engine.base.Engine (7, 2, '2020-12-10 13:18:37.542624')
2020-12-10 13:18:41,557 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-10 13:18:41,558 INFO sqlalchemy.engine.base.Engine (5, 3, '2020-12-10 13:18:38.544037')
2020-12-10 13:18:41,560 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-10 13:18:41,561 INFO sqlalchemy.engine.base.Engine (3, 4, '2020-12-10 13:18:39.544580')
2020-12-10 13:18

In [ ]:
Purchase_query =  select([Purchase.CustomerId,Purchase.ItemId,Purchase.Date])  
display_results(Purchase_query)

2020-12-10 15:08:00,744 INFO sqlalchemy.engine.base.OptionEngine SELECT purchases."CustomerId", purchases."ItemId", purchases."Date" 
FROM purchases
2020-12-10 15:08:00,745 INFO sqlalchemy.engine.base.OptionEngine ()


,CustomerId,ItemId,Date
0,6,3,2020-12-10 13:09:16.352380
1,3,2,2020-12-10 13:09:14.349645
2,4,3,2020-12-10 13:09:15.351005
3,1,0,2020-12-10 13:13:10.713913
4,2,1,2020-12-10 13:13:11.715323
5,3,2,2020-12-10 13:13:12.716779
6,4,3,2020-12-10 13:13:13.718217
7,0,4,2020-12-10 13:13:14.719670
8,1,2,2020-12-10 13:18:36.541021
9,2,2,2020-12-10 13:18:37.542624


При помощи запроса SQLAlchemy вывести все покупки Lisa. Вывести тоже самое при помощи Pandas.

In [ ]:
all_no_filter = session.query(Customer,Purchase,Item).join(Purchase, Purchase.CustomerId==Customer.CustomerId).join(Item, Item.ItemId == Purchase.ItemId)
all_no_filter.filter(Customer.FirstName=='Lisa').all()

2020-12-10 15:08:08,024 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-10 15:08:08,028 INFO sqlalchemy.engine.base.Engine SELECT customers."CustomerId" AS "customers_CustomerId", customers."FirstName" AS "customers_FirstName", customers."LastName" AS "customers_LastName", customers."Company" AS "customers_Company", customers."Address" AS "customers_Address", customers."Phone" AS "customers_Phone", customers."Email" AS "customers_Email", purchases."PurchaseId" AS "purchases_PurchaseId", purchases."ItemId" AS "purchases_ItemId", purchases."CustomerId" AS "purchases_CustomerId", purchases."Date" AS "purchases_Date", items."ItemId" AS "items_ItemId", items."Name" AS "items_Name", items."Price" AS "items_Price" 
FROM customers JOIN purchases ON purchases."CustomerId" = customers."CustomerId" JOIN items ON items."ItemId" = purchases."ItemId" 
WHERE customers."FirstName" = ?
2020-12-10 15:08:08,031 INFO sqlalchemy.engine.base.Engine ('Lisa',)


[(Customer(CustomerId=6, FirstName='Lisa', LastName='White', Company=None, Address='Alenbi 66, Tel Aviv', Phone='062-1234565', Email='lisa@White.com'),
  Purchase(PurchaseId=1, ItemId=3, CustomerId=6, Date=datetime.datetime(2020, 12, 10, 13, 9, 16, 352380)),
  Item(ItemId=3, Name='Audi 3', Price=Decimal('3000.0000000000'))),
 (Customer(CustomerId=6, FirstName='Lisa', LastName='White', Company=None, Address='Alenbi 66, Tel Aviv', Phone='062-1234565', Email='lisa@White.com'),
  Purchase(PurchaseId=13, ItemId=4, CustomerId=6, Date=datetime.datetime(2020, 12, 10, 13, 18, 40, 545947)),
  Item(ItemId=4, Name='Audi 4', Price=Decimal('4000.0000000000')))]

In [172]:
query = """SELECT * FROM customers t1  
join purchases t2
on t1.CustomerId = t2.CustomerId
join items t3
on t2.ItemId = t3.ItemId
where t1.FirstName ='Lisa'
"""
df = pd.read_sql(query, conn)
df.tail()

2020-12-10 16:01:30,764 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("SELECT * FROM customers t1  
join purchases t2
on t1.CustomerId = t2.CustomerId
join items t3
on t2.ItemId = t3.ItemId
where t1.FirstName ='Lisa'
")
2020-12-10 16:01:30,765 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 16:01:30,767 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("SELECT * FROM customers t1  
join purchases t2
on t1.CustomerId = t2.CustomerId
join items t3
on t2.ItemId = t3.ItemId
where t1.FirstName ='Lisa'
")
2020-12-10 16:01:30,771 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 16:01:30,774 INFO sqlalchemy.engine.base.Engine SELECT * FROM customers t1  
join purchases t2
on t1.CustomerId = t2.CustomerId
join items t3
on t2.ItemId = t3.ItemId
where t1.FirstName ='Lisa'

2020-12-10 16:01:30,776 INFO sqlalchemy.engine.base.Engine ()


,CustomerId,FirstName,LastName,Company,Address,Phone,Email,PurchaseId,ItemId,CustomerId,Date,ItemId,Name,Price
0,6,Lisa,White,None,"Alenbi 66, Tel Aviv",062-1234565,lisa@White.com,1,3,6,2020-12-10 13:09:16.352380,3,Audi 3,3000
1,6,Lisa,White,None,"Alenbi 66, Tel Aviv",062-1234565,lisa@White.com,13,4,6,2020-12-10 13:18:40.545947,4,Audi 4,4000



<center><img src='https://github.com/ddvika/Data-Science-School-2020/blob/main/lecture_5/imgs/i-know-sql.jpg?raw=true'></center>